# Static Crawler

## 1. Setting

In [1]:
import pandas as pd

from bs4 import BeautifulSoup
from newspaper import Article

from symcrawl import Crawler, url_to_soup

In [2]:
CATEGORIES = ['정치', '경제', '사회', '세계', '생활/문화', 'IT/과학']
CATEGORY_TO_ID = {
    '정치': '100',
    '경제': '101',
    '사회': '102',
    '생활/문화': '103',
    '세계': '104',
    'IT/과학': '105',
}

In [3]:
date = '20211005'
max_page = 1

## 2. List Crawling

In [4]:
class ListCrawler(Crawler):
    def __init__(self, date, max_page, engine_type='static', use_ray=True, use_tqdm=True):
        super().__init__(engine_type, use_ray, use_tqdm)
        self.date = date
        self.max_page = max_page

    def get_page_url(self, category, page):
        category_id = CATEGORY_TO_ID[category]
        url = f'https://news.naver.com/main/list.naver?mode=LSD&mid=sec&sid1={category_id}&listType=title&date={self.date}&page={page}'
        return url

    def find_max_page(self, category):
        page_url = self.get_page_url(category, 10000)
        soup = url_to_soup(page_url)
        max_page = soup.select('div.paging strong')[0].text
        return min(self.max_page, int(max_page))

    def browse(self):
        urls = []
        for category in CATEGORIES:
            max_page = self.find_max_page(category)
            urls += [self.get_page_url(category, p) for p in range(1, max_page+1)]
        return urls

    def parse(self, html):
        soup = BeautifulSoup(html, 'html.parser')
        articles = soup.select('div.list_body ul.type02 a')
        titles = [i.text for i in articles]
        urls = [i.get('href') for i in articles]
        authors = [i.text for i in soup.select('div.list_body ul.type02 span.writing')]
        return [{'title': t, 'author': a, 'url':u} for t, a, u in zip(titles, authors, urls)]

In [5]:
list_crawler = ListCrawler(date, max_page, use_ray=False)

In [7]:
meta = list_crawler.crawl()
meta.head()

request:   0%|          | 0/6 [00:00<?, ?it/s]

parse:   0%|          | 0/6 [00:00<?, ?it/s]

,title,author,url
0,홍남기 “11월 가스요금 동결합니다”,서울경제,https://news.naver.com/main/read.naver?mode=LS...
1,"""文경제 학사경고 수준""…부총리에 몰아친 '부동산 포화'(종합)",뉴스1,https://news.naver.com/main/read.naver?mode=LS...
2,"이재명, 2004년 음주운전 당시 혈중알코올 0.158%…면허취소 수준",KBS,https://news.naver.com/main/read.naver?mode=LS...
3,"""화천대유했나"" vs ""윤석열 장모는?""…기재위에 '대장동 유탄'",머니투데이,https://news.naver.com/main/read.naver?mode=LS...
4,"박범계 ""고발사주, 중대 사건…공수처 이첩 아닌 이송이라 썼다""(종합)",뉴스1,https://news.naver.com/main/read.naver?mode=LS...


## 3. Article Crawling

In [12]:
class ArticleCrawler(Crawler):
    def __init__(self, meta, engine_type='static', use_ray=False, use_tqdm=True):
        super().__init__(engine_type, use_ray, use_tqdm)
        self.meta = meta
    
    def browse(self):
        return self.meta['url'].to_list()

    def parse(self, html):
        article = Article('', language='ko', fetch_images=False)
        article.download(html)
        article.parse()
        text = article.text
        return {'text': text}

    def postprocess(self, data):
        data = pd.concat([meta, data], axis=1)
        return data

In [15]:
article_crawler = ArticleCrawler(meta)

In [16]:
data = article_crawler.crawl()
data.head()

request:   0%|          | 0/300 [00:00<?, ?it/s]

parse:   0%|          | 0/300 [00:00<?, ?it/s]

,title,author,url,text
0,홍남기 “11월 가스요금 동결합니다”,서울경제,https://news.naver.com/main/read.naver?mode=LS...,홍남기 부총리 겸 기획재정부 장관이 5일 국회에서 열린 기획재정위원회의 기획재정부 ...
1,"""文경제 학사경고 수준""…부총리에 몰아친 '부동산 포화'(종합)",뉴스1,https://news.naver.com/main/read.naver?mode=LS...,5일 국회 기획재정위원회 국정감사에 출석한 홍남기 경제부총리. 2021.10.5/뉴...
2,"이재명, 2004년 음주운전 당시 혈중알코올 0.158%…면허취소 수준",KBS,https://news.naver.com/main/read.naver?mode=LS...,더불어민주당 이재명 대선 경선 후보가 2004년에 음주운전으로 적발될 당시 혈중알코...
3,"""화천대유했나"" vs ""윤석열 장모는?""…기재위에 '대장동 유탄'",머니투데이,https://news.naver.com/main/read.naver?mode=LS...,홍남기 경제부총리 겸 기획재정부 장관이 5일 오전 서울 여의도 국회에서 기획재정위원...
4,"박범계 ""고발사주, 중대 사건…공수처 이첩 아닌 이송이라 썼다""(종합)",뉴스1,https://news.naver.com/main/read.naver?mode=LS...,박범계 법무부 장관이 5일 서울 여의도 국회에서 열린 법제사법위원회의 법무부·대한법...
